In [27]:
import sys
sys.path.append('../')

import cqt
import cqt.datagen as dg
import cqt.dbutility.dbutility as db

import cqt.ledger.ledger as ledger

from cqt.strats.strategy import Strategy
import cqt.strats.strategy_double_dip as sdd
import cqt.strats.strategy_long_short_average as sma

import cqt.env.mkt_env as mkt_env
import cqt.env.mkt_env_spot as envspot
import pandas as pd
import numpy as np
import pickle

from datetime import datetime
from datetime import timedelta
import matplotlib.pyplot as plt

In [28]:
TODAY = datetime.now()
BACK_TEST_START_DATE = datetime(2018,1,1)

In [29]:
def getAssetData(assetName, fromDate, toDate):
    assetNameToTableNameDict = {
        'btc': 'coinapi-ohlcv-COINBASE_SPOT_BTC_USD-1DAY'
    }
    fromDateStr = fromDate.strftime('%Y-%M-%d')
    toDateStr = toDate.strftime('%Y-%m-%d')
    tblName = assetNameToTableNameDict[assetName]
    df=db.get_from_db(tblName,from_date=fromDateStr,to_date=toDateStr)
    return df

In [30]:
today = TODAY
backTestStartDate = BACK_TEST_START_DATE
assetName = 'btc'

In [31]:
# assetNameToTableNameDict = {
#         'btc': 'coinapi-ohlcv-COINBASE_SPOT_BTC_USD-1DAY'
#     }
# fromDate = backTestStartDate
# toDate = today
# tblName = assetNameToTableNameDict[assetName]
# fromDateStr = fromDate.strftime('%Y-%M-%d')
# toDateStr = toDate.strftime('%Y-%m-%d')

In [32]:
df = getAssetData(assetName, backTestStartDate, today)
sec = envspot.MktEnvSpot(assetName, df.get_index_data())
myEnv = mkt_env.MktEnv([])
myEnv.insert_section(sec)

In [36]:
#
cash = 100000
assets = {'btc': 0, 'eth': 0}
portfolio = ledger.Ledger(assets, cash)

In [37]:
# Long/Short Average

ls_rule = {'method' : 'moving_average', 'window_size' : [30, 90], 'tolerance_up' : 0.0, 'tolerance_down' : 0.0}
strats = sma.StrategySimpleMA(myEnv, portfolio, ls_rule)
report = strats.back_testing(backTestStartDate)

IndexError: single positional indexer is out-of-bounds